In [0]:
pip install opencv-python

In [0]:
pip install ultralytics

In [0]:
import os
import cv2
from ultralytics import YOLO
import yaml
from PIL import Image as PILImage

def is_valid_image(path):
    try:
        with PILImage.open(path) as img:
            img.verify()
        return True
    except Exception as e:
        print(f"Vaizdas neįskaitomas: {path}, klaida: {e}")
        return False

def blur_local_image(path, yolo_model, idx2class, classes_to_blur, blur_intensity):
    print(f"Processing: {path}")

    if not is_valid_image(path):
        print(f"Netinkamas failas: {path}")
        return

    hat = yolo_model.predict(
        path,
        conf=0.1,
        iou=0.4,
    )
    boxes = hat[0].boxes
    labels = boxes.cls.cpu().numpy()
    bboxes = boxes.xyxy

    label_names = [idx2class[int(label)] for label in labels]

    img_cv = cv2.imread(path)
    blurred_count = 0

    for i, box in enumerate(bboxes):
        if label_names[i] in classes_to_blur:
            x0, y0, x1, y1 = map(int, box)
            img_cv[y0:y1, x0:x1] = cv2.blur(img_cv[y0:y1, x0:x1], (blur_intensity, blur_intensity))
            blurred_count += 1

    output_path = path.replace(".", "_blurred.")
    cv2.imwrite(output_path, img_cv)
    print(f"Išsaugota: {output_path} (blurred objektai: {blurred_count})")

if __name__ == "__main__":
    with open("configuration.yml", "r") as f:
        config = yaml.load(f, Loader=yaml.FullLoader)

    idx2class = config["CLASS_NAMES"]
    classes_to_blur = config["CLASSES_TO_BLURR"]
    blur_intensity = int(config["BLURR_INTENSITY"])
    yolo_model = YOLO(config["YOLO_MODEL"])


    test_files = [
    ]

    for path in test_files:
        if os.path.exists(path):
            blur_local_image(path, yolo_model, idx2class, classes_to_blur, blur_intensity)
        else:
            print(f"Failas nerastas: {path}")
